In [3]:
import numpy as np                 # linear algebra
import pandas as pd                # data processing
import matplotlib.pyplot as plt    # basic plotting library
import seaborn as sns              # more advanced visual plotting library

In [4]:
headers = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "fClass"]
df = pd.read_csv('magic04.data', names=headers)
mapping = {'g': 1, 'h': 0}
df["fClass"] = df["fClass"].map(mapping)
df.describe()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,fClass
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026,0.648370
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787,0.477492
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600,0.000000
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250,0.000000
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450,1.000000
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825,1.000000
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000,1.000000


In [ ]:
df_rescaled = pd.DataFrame()
headers = ["fLength", "fWidth", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fDist","fAlpha", "fSize","fClass"]
rescaled_headers = [ "rescaled_length", "rescaled_width","rescaled_conc","rescaled_conc1"  ,"rescaled_asym","rescaled_M3long" ,"rescaled_M3trans","rescaled_dist" ,"alpha","size","rad"]
for i in range(0,8):
    df_rescaled.insert(i, rescaled_headers[i] ,df[headers[i]]/df["fSize"])
df_rescaled.insert(8, "alpha" ,df["fAlpha"])
df_rescaled.insert(9, "size" ,df["fSize"])
df_rescaled.insert(10, "rad" ,df["fClass"])
df_rescaled.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
print(len(df[df['fAlpha'] == 0]))
print(len(df[df['fAlpha'] == 90]))
print(len(df[df['fSize'] == 0]))
print(len(df[df['fLength'] == 0]))
print(len(df[df['fWidth'] == 0]))

In [ ]:
df[df['fWidth'] < 1].hist(column='fWidth', bins=100)
df[df['fWidth'] == 0].hist(column='fSize', bins=100)

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap(data=df.corr(),annot=True,cmap="bone",linewidths=1,fmt=".2f",linecolor="gray")
plt.title("Correlation Map",fontsize=20)
plt.tight_layout()
plt.show()      # lightest and darkest cells are most correlated ones

In [ ]:
sns.pairplot(data=df,
             palette="husl",
             hue="fClass",
             vars=headers)
plt.suptitle("PairPlot of Data Without Std. Dev. Fields",fontsize=18)
plt.tight_layout()
plt.show()   

In [9]:
# df.drop(df[df.fWidth == 0].index, inplace=True) # Drops rows where fWidth == 0

"""
Y = df.fClass.values
df.drop(["fClass"],axis=1,inplace=True) # Drops column fClass
X = df.values
"""

Y = df["fClass"].values
X = df.drop(["fClass"],axis=1).values

# Y = df_rescaled["rad"].values
# X = df_rescaled.drop(["rad"],axis=1).values

In [10]:
# Shuffle the DataFrame rows, train_test_split already does that if we use random_state
# df = df.sample(frac=1)

In [11]:
# Scaling, we might use something different
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.05,random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(18069, 10)
(18069,)
(951, 10)
(951,)


In [7]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(solver="sag",C=0.1,penalty="l2", tol = 1e-20, max_iter = 100, random_state = 100)
lr_model.fit(x_train,y_train)
lr_score = lr_model.score(x_test,y_test)
lr_pred = lr_model.predict(x_test)
print(lr_score)
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test,lr_model.predict_proba(x_test)[:, 1])
print(roc_score)

0.8002103049421662
0.8407831891320519


C:\Users\basti\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
from sklearn.metrics import confusion_matrix
cm_lr = confusion_matrix(y_test,lr_pred)

plt.figure(figsize=(24,12))
plt.suptitle("Confusion Matrixes",fontsize=24)
plt.subplot(1,2,1)
plt.title("Logistic Regression Confusion Matrix")
sns.heatmap(cm_lr,cbar=False,annot=True,cmap="CMRmap_r",fmt="d")

plt.show()

In [12]:
import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler

In [13]:
# look for theory about no of parameters vs data rows
# test different activation functions
# check classifiers
# check general pourpose layers instead of dense ones
# check loss function
# check optimizer for loss function

model = Sequential()
model.add(Dense(12, activation=tf.nn.relu, input_shape=(10,)))
model.add(Dense(6, activation=tf.nn.sigmoid))
model.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

#non-working attempt at a custom loss function as in: https://towardsdatascience.com/explicit-auc-maximization-70beef6db14e
# def custom_loss(y_true, y_pred):
#     # calculate loss, using y_pred
#     alpha = tf.sigmoid(y_pred @ tf.transpose(y_pred))
#     beta = np.maximum(y_true @ np.ones(y_true.shape).T - np.ones(y_true.shape) @ y_true.T)
#     loss = - tf.reduce_mean(alpha * beta, 0)
#     return loss
# model.compile(loss=custom_loss, optimizer='adam')

print("Number of weights after calling the model:", len(model.weights))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                132       
                                                                 
 dense_4 (Dense)             (None, 6)                 78        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 217
Trainable params: 217
Non-trainable params: 0
_________________________________________________________________
Number of weights after calling the model: 6


In [14]:
dnn_history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=10)

Epoch 1/10
1807/1807 [==============================] - 3s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 2/10
1807/1807 [==============================] - 3s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 3/10
1807/1807 [==============================] - 3s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 4/10
1807/1807 [==============================] - 2s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 5/10
1807/1807 [==============================] - 2s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 6/10
1807/1807 [==============================] - 2s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_accuracy: 0.3449
Epoch 7/10
1807/1807 [==============================] - 2s 1ms/step - loss: 9.9956 - accuracy: 0.3520 - val_loss: 10.1049 - val_ac

In [ ]:
plt.plot(dnn_history.history['accuracy'])
plt.plot(dnn_history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training data', 'Validation data'], loc='lower right')
plt.show()

In [25]:
dnn_score = model.evaluate(x_test, y_test)
dnn_pred = model.predict(x_test)
print(model.metrics_names)
print("dnn_score: ",dnn_score)

from sklearn.metrics import roc_auc_score
y_score = model.predict(x_test)
roc_score = roc_auc_score(y_test,y_score)
print("roc score: ", roc_score)

30/30 [==============================] - 0s 1ms/step - loss: 10.1049 - accuracy: 0.3449
['loss', 'accuracy']
dnn_score:  [10.104883193969727, 0.34490010142326355]
roc score:  0.5115834083701992
